In [ ]:
!pip install langchain openai duckduckgo-search youtube_search wikipedia langchainhub

In [ ]:
!pip install gradio==3.43.1

In [ ]:
import getpass

# Prompt the user for input without showing what they type
open_api_key = getpass.getpass("Enter your OpenAI API key: ")

# Now you can use the api_key variable to store the API key securely


Enter your OpenAI API key: ··········


In [ ]:
# Ensure you take care of imports, setting your OpenAI key, etc.
#%%capture

import os
import getpass

import getpass

# Prompt the user for input without showing what they type
api_key = getpass.getpass("Enter your OpenAI API key: ")
from langchain.agents import Tool, AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.agents import AgentExecutor
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description
import gradio as gr


Enter your OpenAI API key: ··········


In [ ]:
!pip install requests

In [ ]:
import requests

def fetch_and_process_disasters():
    # Hardcoded input text
    input_text = "Disasters for zip code 99083"

    def fetch_disasters():
        url = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return data['DisasterDeclarationsSummaries']
        else:
            print(f"Failed to fetch data: {response.status_code}")
            return []

    def filter_disasters_by_place_code(disasters, place_code):
        return [disaster for disaster in disasters if str(disaster.get('placeCode', '')) == str(place_code)]

    if "zip code" in input_text:
        zip_code = input_text.split("zip code")[1].strip()
        all_disasters = fetch_disasters()
        relevant_disasters = filter_disasters_by_place_code(all_disasters, zip_code)

        if not relevant_disasters:
            return "No disasters found for this place code."

        output = []
        for disaster in relevant_disasters:
            disaster_info = f"Title: {disaster.get('declarationTitle', 'N/A')}, " \
                            f"State: {disaster.get('state', 'N/A')}, " \
                            f"Begin Date: {disaster.get('incidentBeginDate', 'N/A')}, " \
                            f"End Date: {disaster.get('incidentEndDate', 'N/A')}"
            output.append(disaster_info)
        return "\n".join(output)
    else:
        return "Unrecognized input. Please specify your request more clearly."

# Call the function and print its response
response = fetch_and_process_disasters()
print(response)


Title: ALISAL FIRE, State: CA, Begin Date: 2021-10-12T00:00:00.000Z, End Date: 2021-11-17T00:00:00.000Z
Title: HOLIDAY FIRE, State: CA, Begin Date: 2018-07-07T00:00:00.000Z, End Date: 2018-07-10T00:00:00.000Z
Title: THOMAS FIRE, State: CA, Begin Date: 2017-12-04T00:00:00.000Z, End Date: None
Title: TATUM GULF FIRE, State: GA, Begin Date: 2016-11-13T00:00:00.000Z, End Date: 2016-12-02T00:00:00.000Z
Title: GUTHRIE FIRE, State: OK, Begin Date: 2014-05-05T00:00:00.000Z, End Date: None
Title: SEVERE STORMS, STRAIGHT-LINE WINDS, AND TORNADO, State: TN, Begin Date: 2023-03-01T00:00:00.000Z, End Date: 2023-03-03T00:00:00.000Z
Title: SEVERE STORMS, STRAIGHT-LINE WINDS, TORNADOES, FLOODING, LANDSLIDES, AND MUDSLIDES, State: KY, Begin Date: 2023-03-03T00:00:00.000Z, End Date: 2023-03-04T00:00:00.000Z
Title: SEVERE WINTER STORMS, FLOODING, LANDSLIDES, AND MUDSLIDES, State: CA, Begin Date: 2022-12-27T00:00:00.000Z, End Date: 2023-01-31T00:00:00.000Z
Title: HURRICANE NICOLE, State: FL, Begin Date: 2

In [ ]:
fema_tool = Tool(
    name='Random number',
    func= fetch_and_process_disasters,
    description="Useful for fema searching"
)

In [ ]:
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI
import os

os.environ["OPENAI_API_KEY"] = ""
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)
tools = [fema_tool]
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
# conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)


# create our agent
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=turbo_llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)

In [ ]:
import requests
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool  # Assuming Tool is correctly imported from LangChain or defined elsewhere
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
import os

# Original function adapted to accept zip code as an argument
def fetch_and_process_disasters(zip_code):
    def fetch_disasters():
        url = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return data['DisasterDeclarationsSummaries']
        else:
            print(f"Failed to fetch data: {response.status_code}")
            return []

    def filter_disasters_by_place_code(disasters, place_code):
        return [disaster for disaster in disasters if str(disaster.get('placeCode', '')) == str(place_code)]

    all_disasters = fetch_disasters()
    relevant_disasters = filter_disasters_by_place_code(all_disasters, zip_code)

    if not relevant_disasters:
        return "No disasters found for this place code."

    output = []
    for disaster in relevant_disasters:
        disaster_info = f"Title: {disaster.get('declarationTitle', 'N/A')}, " \
                        f"State: {disaster.get('state', 'N/A')}, " \
                        f"Begin Date: {disaster.get('incidentBeginDate', 'N/A')}, " \
                        f"End Date: {disaster.get('incidentEndDate', 'N/A')}"
        output.append(disaster_info)
    return "\n".join(output)

# Wrapper function for compatibility with LangChain
def fetch_and_process_disasters_wrapper(input_text):
    # Directly use the input text as the zip code for simplicity
    return fetch_and_process_disasters(input_text)

# Set up the conversational agent environment
os.environ["OPENAI_API_KEY"] = ""
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

# Create the FEMA search tool using the wrapper function
fema_tool = Tool(
    name='FEMA Search',
    func=fetch_and_process_disasters,  # Use the wrapper function here
    description="Useful for FEMA searching using zip code entered"
)

# Set up conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)

# Initialize the conversational agent
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=[fema_tool],
    llm=turbo_llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)

# Assuming the correct method to invoke the agent, following LangChain's API updates
response = conversational_agent.invoke("99083")
print(response)




> Entering new AgentExecutor chain...
```json
{
    "action": "FEMA Search",
    "action_input": "99083"
}
```
Observation: Title: ALISAL FIRE, State: CA, Begin Date: 2021-10-12T00:00:00.000Z, End Date: 2021-11-17T00:00:00.000Z
Title: HOLIDAY FIRE, State: CA, Begin Date: 2018-07-07T00:00:00.000Z, End Date: 2018-07-10T00:00:00.000Z
Title: THOMAS FIRE, State: CA, Begin Date: 2017-12-04T00:00:00.000Z, End Date: None
Title: TATUM GULF FIRE, State: GA, Begin Date: 2016-11-13T00:00:00.000Z, End Date: 2016-12-02T00:00:00.000Z
Title: GUTHRIE FIRE, State: OK, Begin Date: 2014-05-05T00:00:00.000Z, End Date: None
Title: SEVERE STORMS, STRAIGHT-LINE WINDS, AND TORNADO, State: TN, Begin Date: 2023-03-01T00:00:00.000Z, End Date: 2023-03-03T00:00:00.000Z
Title: SEVERE STORMS, STRAIGHT-LINE WINDS, TORNADOES, FLOODING, LANDSLIDES, AND MUDSLIDES, State: KY, Begin Date: 2023-03-03T00:00:00.000Z, End Date: 2023-03-04T00:00:00.000Z
Title: SEVERE WINTER STORMS, FLOODING, LANDSLIDES, AND MUDSLIDES, State: 

In [ ]:
# Create toolkit
search = DuckDuckGoSearchAPIWrapper()

search_tool = Tool(name="Current Search",
                   func=search.run,
                   description="Useful when you need to answer questions about nouns, current events or the current state of the world."
                   )

tools = [search_tool, fema_tool ]

# Set up memory
memory = ConversationBufferMemory(memory_key="chat_history")

# Initialize LLM
llm = ChatOpenAI(model = "gpt-3.5-turbo", temperature=0, openai_api_key="" )
print('llm initialized')

# Initialize agent
agent_chain = initialize_agent(tools,
                               llm,
                               agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
                               memory=memory,
                               verbose=True)

# Run queries
# Here is an example query:
agent_chain.run(input="What it do, nephew!")
agent_prompt = hub.pull("hwchase17/react-chat")

# Setting up an agent using LCEL
prompt = agent_prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)
llm_with_stop = llm.bind(stop=["\nObservation"])

# Set up the agent using LCEL
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

memory = ConversationBufferMemory(memory_key="chat_history")

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

# # Here are some example queries:
agent_executor.invoke({"input": " I live in zip code 99083. Can you tell if there are any disasters around me in the past 2 years?"})["output"]
# agent_executor.invoke({"input": "What did I just ask you about?"})["output"]
# agent_executor.invoke({"input": "Will it look like a white Christmas there?"})["output"]


llm initialized


> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Thought: Do I need to use a tool? No
AI: Hey there! How can I assist you today?

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: FEMA Search
Action Input: 99083Title: ALISAL FIRE, State: CA, Begin Date: 2021-10-12T00:00:00.000Z, End Date: 2021-11-17T00:00:00.000Z
Title: HOLIDAY FIRE, State: CA, Begin Date: 2018-07-07T00:00:00.000Z, End Date: 2018-07-10T00:00:00.000Z
Title: THOMAS FIRE, State: CA, Begin Date: 2017-12-04T00:00:00.000Z, End Date: None
Title: TATUM GULF FIRE, State: GA, Begin Date: 2016-11-13T00:00:00.000Z, End Date: 2016-12-02T00:00:00.000Z
Title: GUTHRIE FIRE, State: OK, Begin Date: 2014-05-05T00:00:00.000Z, End Date: None
Title: SEVERE STORMS, STRAIGHT-LINE WINDS, AND TORNADO, State: TN, Begin Date: 2023-03-01T00:00:00.000Z, End Date: 2023-03-03T00:00:00.000Z
Title: SEVERE STORMS, STRAIGHT-LINE WINDS, TORNADOES, FLOODING, LANDSLIDES, AND MUDSLIDES, State: KY, Begin Date: 2023-03-03T00:00:00.000Z, End Date: 2023-03-

'Based on the FEMA search results, there have been several disasters around your area in the past 2 years, including the Alisal Fire and the Holiday Fire in California.'

In [ ]:
import requests
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool  # Assuming Tool is correctly imported from LangChain or defined elsewhere
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
import os

# Original function adapted to accept zip code as an argument
def fetch_and_process_disasters(zip_code):
    def fetch_disasters():
        url = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return data['DisasterDeclarationsSummaries']
        else:
            print(f"Failed to fetch data: {response.status_code}")
            return []

    def filter_disasters_by_place_code(disasters, place_code):
        return [disaster for disaster in disasters if str(disaster.get('placeCode', '')) == str(place_code)]

    all_disasters = fetch_disasters()
    relevant_disasters = filter_disasters_by_place_code(all_disasters, zip_code)

    if not relevant_disasters:
        return "No disasters found for this place code."

    output = []
    for disaster in relevant_disasters:
        disaster_info = f"Title: {disaster.get('declarationTitle', 'N/A')}, " \
                        f"State: {disaster.get('state', 'N/A')}, " \
                        f"Begin Date: {disaster.get('incidentBeginDate', 'N/A')}, " \
                        f"End Date: {disaster.get('incidentEndDate', 'N/A')}"
        output.append(disaster_info)
    return "\n".join(output)

# Wrapper function for compatibility with LangChain
def fetch_and_process_disasters_wrapper(input_text):
    # Directly use the input text as the zip code for simplicity
    return fetch_and_process_disasters(input_text)

# Set up the conversational agent environment
os.environ["OPENAI_API_KEY"] = ""
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

# Create the FEMA search tool using the wrapper function
fema_tool = Tool(
    name='FEMA Search',
    func=fetch_and_process_disasters,  # Use the wrapper function here
    description="Useful for FEMA searching using zip code entered"
)

# Set up conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)

# Initialize the conversational agent
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=[fema_tool],
    llm=turbo_llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)

# Assuming the correct method to invoke the agent, following LangChain's API updates
response = conversational_agent.invoke("99083")
print(response)




> Entering new AgentExecutor chain...
```json
{
    "action": "FEMA Search",
    "action_input": "99083"
}
```
Observation: Title: ALISAL FIRE, State: CA, Begin Date: 2021-10-12T00:00:00.000Z, End Date: 2021-11-17T00:00:00.000Z
Title: HOLIDAY FIRE, State: CA, Begin Date: 2018-07-07T00:00:00.000Z, End Date: 2018-07-10T00:00:00.000Z
Title: THOMAS FIRE, State: CA, Begin Date: 2017-12-04T00:00:00.000Z, End Date: None
Title: TATUM GULF FIRE, State: GA, Begin Date: 2016-11-13T00:00:00.000Z, End Date: 2016-12-02T00:00:00.000Z
Title: GUTHRIE FIRE, State: OK, Begin Date: 2014-05-05T00:00:00.000Z, End Date: None
Title: SEVERE STORMS, STRAIGHT-LINE WINDS, AND TORNADO, State: TN, Begin Date: 2023-03-01T00:00:00.000Z, End Date: 2023-03-03T00:00:00.000Z
Title: SEVERE STORMS, STRAIGHT-LINE WINDS, TORNADOES, FLOODING, LANDSLIDES, AND MUDSLIDES, State: KY, Begin Date: 2023-03-03T00:00:00.000Z, End Date: 2023-03-04T00:00:00.000Z
Title: SEVERE WINTER STORMS, FLOODING, LANDSLIDES, AND MUDSLIDES, State: 

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 25.3 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import torch

class TextLoader:
    def __init__(self, file_path):
        self.file_path = file_path

    def load(self):
        with open(self.file_path, 'r') as file:
            documents = file.readlines()
        return [{"page_content": doc.strip()} for doc in documents]

def generate_augmented_answer(query):
    # Load text documents
    loader = TextLoader("/content/Disaster_RAG (1).txt")
    documents = loader.load()
    text_documents = [doc["page_content"] for doc in documents]

    # Embed the documents
    model = SentenceTransformer("sentence-transformers/all-MiniLM-l6-v2")
    embeddings = model.encode(text_documents, convert_to_tensor=True)

    # Create FAISS index
    index = faiss.IndexFlatL2(embeddings.shape[1])
    faiss.normalize_L2(embeddings.numpy())
    index.add(embeddings.numpy())

    # Embed the query
    query_embedding = model.encode([query], convert_to_tensor=True)
    faiss.normalize_L2(query_embedding.numpy())

    # Perform similarity search
    _, doc_indices = index.search(query_embedding.numpy(), k=5)
    similar_docs = [text_documents[i] for i in doc_indices[0]]

    # Augment the query with information from similar documents
    augmented_query = "Considering relevant information from retrieved documents: " + "\n  ".join(similar_docs) + ", what is the answer to the question: " + query

    # Query the text generation model
    # from transformers import pipeline
    # generator = pipeline("text-generation", model="google/flan-t5-base", tokenizer="google/flan-t5-base", framework="pt")
    # augmented_answers = generator(augmented_query, max_length=256, temperature=1)

    return augmented_query

# Example usage
query = "There is a wild fire what do I do ?"
augmented_answer = generate_augmented_answer(query)
print(augmented_answer)


README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Considering relevant information from retrieved documents: Question: "What to do if a wildfire is approaching my area in California?"
  * Answer: "Evacuate immediately if advised by authorities. Pack your emergency kit, including essential documents, medications, and personal items. Use N95 masks to protect from smoke inhalation. Shut off gas at the meter and leave exterior lights on for visibility. Follow designated evacuation routes and avoid back roads. For wildfire updates and evacuation orders, call 213-456-7890."
  Answer: "If you smell gas, do not turn on lights or use anything that might spark a fire. Open windows and doors to ventilate the area. Evacuate everyone from the building immediately and call the gas leak hotline at 234-567-8901 from a safe distance. Do not return until it has been declared safe by professionals."
  Question: "What to do if I encounter wildlife displaced by a disaster?"
  Question: "How to respond to air quality alerts in California due to wildfires?"

In [ ]:
os.environ["OPENAI_API_KEY"] = ""
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

# Create the FEMA search tool using the wrapper function
fema_tool = Tool(
    name='RAG Search',
    func=generate_augmented_answer,  # Use the wrapper function here
    description="Use this information to answer question related to information for Natural Disasters, provide details such as the phone numbers or location details ",
)

# Set up conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)

# Initialize the conversational agent
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=[fema_tool],
    llm=turbo_llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)

# Assuming the correct method to invoke the agent, following LangChain's API updates
response = conversational_agent.invoke("There is a wild fire what do I do ?")
print(response)



> Entering new AgentExecutor chain...
```json
{
    "action": "RAG Search",
    "action_input": "wildfire safety tips"
}
```
Observation: Considering relevant information from retrieved documents: * Answer: "Evacuate immediately if advised by authorities. Pack your emergency kit, including essential documents, medications, and personal items. Use N95 masks to protect from smoke inhalation. Shut off gas at the meter and leave exterior lights on for visibility. Follow designated evacuation routes and avoid back roads. For wildfire updates and evacuation orders, call 213-456-7890."
  Question: "What to do if a wildfire is approaching my area in California?"
  Question: "How to respond to air quality alerts in California due to wildfires?"
  Question: "How to stay safe during extreme heat waves in California?"
  Question: "What precautions should I take during an extreme cold or winter storm?", what is the answer to the question: wildfire safety tips
Thought:```json
{
    "action": "Fina

In [ ]:
def ask_for_rating(input_text):
    return "On a scale of 1 to 10, how adequately do you feel your needs were met during the disaster response?"

# Define the function to handle the rating response
def handle_rating_response(input_text):
    # You can process the rating response here
    return "Thank you for your feedback!"

# Example of executing the function with a string input
response = ask_for_rating("Bye")
print(response)

# Assuming Tool and ConversationBufferWindowMemory are part of the same or another library
rate_tool = Tool(
    name='Rate Experience',
    func=handle_rating_response,  # Use the function to handle rating response
    description="Handle rating response"
)

memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)

# Initialize the conversational agent with both rating and rating handling tools
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=[rate_tool],
    llm=turbo_llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)

# Execute the agent with a given input
response = conversational_agent.invoke("Bye")

# Print the initial response from the agent
print(response)

# Ask for rating after providing the response
rating_prompt = ask_for_rating(response['output'])
print(rating_prompt)

# Assume rating is provided by the user
rating_response = input("Please provide your rating: ")

# Handle the rating response using the appropriate tool
rating_response_output = conversational_agent.invoke(rating_response)
print(rating_response_output)


On a scale of 1 to 10, how adequately do you feel your needs were met during the disaster response?


> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Goodbye! If you have any more questions in the future, feel free to ask."
}
```

> Finished chain.
{'input': 'Bye', 'chat_history': [], 'output': 'Goodbye! If you have any more questions in the future, feel free to ask.'}
On a scale of 1 to 10, how adequately do you feel your needs were met during the disaster response?


KeyboardInterrupt: Interrupted by user

In [ ]:
def ask_for_rating(input_text):

        return "On a scale of 1 to 10, how adequately do you feel your needs were met during the disaster response?"



# Example of executing the function with a string input
response = ask_for_rating("thats all")
print(response)

On a scale of 1 to 10, how adequately do you feel your needs were met during the disaster response?


In [ ]:
rate_tool = Tool(
    name='Rate Experience',
    func=ask_for_rating,  # Use the function to handle rating response
    description="when user says, thats all or thanks it for helping, ask for rating response"
)

memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)

# Initialize the conversational agent with both rating and rating handling tools
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=[rate_tool],
    llm=turbo_llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)


# Assuming the correct method to invoke the agent, following LangChain's API updates
response = conversational_agent.invoke("that's all")
print(response)




> Entering new AgentExecutor chain...
```json
{
    "action": "Rate Experience",
    "action_input": "How would you rate your experience with the assistance provided?"
}
```
Observation: On a scale of 1 to 10, how adequately do you feel your needs were met during the disaster response?
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The response to your last comment is: On a scale of 1 to 10, how adequately do you feel your needs were met during the disaster response?"
}
```

> Finished chain.
{'input': "that's all", 'chat_history': [], 'output': 'The response to your last comment is: On a scale of 1 to 10, how adequately do you feel your needs were met during the disaster response?'}
